In [2]:
import pandas as pd
import numpy as np

In [6]:
df=pd.read_csv('selected_features_with_churn.csv')
df.head()

,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,has_gas,nb_prod_act,num_years_antig,pow_max,days_to_end,days_since_modif,days_to_renewal,last_month_vs_avg,gas_to_elec_ratio,margin_ratio,profitability_index,is_high_power,is_low_usage,latest_price_off_peak_fix,churn
0,54946,0,0.00,0.0,1.78,1,2,3,43.648,-199,227,-358,0.000000,54946.0,1.0,678.990000,1,1,44.266930,1
1,0,0,189.95,0.0,16.27,0,1,6,13.800,-123,2566,-365,0.000000,0.0,1.0,0.004053,0,1,44.444710,0
2,0,0,47.96,0.0,38.72,0,1,6,13.856,-259,2192,-365,0.000000,0.0,1.0,0.012110,0,1,44.444710,0
3,0,0,240.04,0.0,19.83,0,1,6,13.200,-276,2192,-365,0.000000,0.0,1.0,0.016063,0,1,44.444710,0
4,0,526,445.75,0.0,131.73,0,1,6,19.800,-299,2245,-364,1.426441,0.0,1.0,0.010840,1,1,40.728885,0


In [7]:
df.shape

(14606, 20)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   cons_gas_12m               14606 non-null  int64  
 1   cons_last_month            14606 non-null  int64  
 2   forecast_cons_12m          14606 non-null  float64
 3   forecast_discount_energy   14606 non-null  float64
 4   forecast_meter_rent_12m    14606 non-null  float64
 5   has_gas                    14606 non-null  int64  
 6   nb_prod_act                14606 non-null  int64  
 7   num_years_antig            14606 non-null  int64  
 8   pow_max                    14606 non-null  float64
 9   days_to_end                14606 non-null  int64  
 10  days_since_modif           14606 non-null  int64  
 11  days_to_renewal            14606 non-null  int64  
 12  last_month_vs_avg          14606 non-null  float64
 13  gas_to_elec_ratio          14606 non-null  flo

In [20]:
df['churn'].value_counts()

churn
0    13187
1     1419
Name: count, dtype: int64

In [9]:
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [32]:
X = df.drop(columns=['churn'])
y = df['churn']

# Normal (non-churn) = 0
X_normal = X[y == 0]

print("Normal samples (churn = 0):", X_normal.shape[0])
print("Churn samples (churn = 1):", (y == 1).sum())

Normal samples (churn = 0): 13187
Churn samples (churn = 1): 1419


In [33]:
scaler = StandardScaler()
X_normal_scaled = scaler.fit_transform(X_normal.values.astype(np.float32))

# We'll also need ALL data scaled later for scoring
X_all_scaled = scaler.transform(X.values.astype(np.float32))

In [34]:
X_normal_tensor = torch.tensor(X_normal_scaled, dtype=torch.float32)
normal_dataset = TensorDataset(X_normal_tensor)
normal_loader = DataLoader(normal_dataset, batch_size=128, shuffle=True)

input_dim = X.shape[1]  # number of features (19 in your case)
hidden_dim = 16
latent_dim = 8

In [35]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        # Decoder
        self.fc_dec1 = nn.Linear(latent_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()

    def encode(self, x):
        h = self.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.relu(self.fc_dec1(z))
        return self.fc_out(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE(input_dim, hidden_dim, latent_dim).to(device)

In [37]:
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
recon_loss_fn = nn.MSELoss(reduction="sum")

In [38]:
def vae_loss(x, x_recon, mu, logvar):
    recon_loss = recon_loss_fn(x_recon, x)
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss

In [39]:
n_epochs = 30

vae.train()
for epoch in range(1, n_epochs + 1):
    total_loss = 0.0
    for (xb,) in normal_loader:
        xb = xb.to(device)

        optimizer.zero_grad()
        x_recon, mu, logvar = vae(xb)
        loss = vae_loss(xb, x_recon, mu, logvar)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(normal_dataset)
    print(f"Epoch {epoch}/{n_epochs} - VAE train loss (normal only): {avg_loss:.4f}")

Epoch 1/30 - VAE train loss (normal only): 24.5324
Epoch 2/30 - VAE train loss (normal only): 20.5104
Epoch 3/30 - VAE train loss (normal only): 19.4724
Epoch 4/30 - VAE train loss (normal only): 18.5597
Epoch 5/30 - VAE train loss (normal only): 18.0361
Epoch 6/30 - VAE train loss (normal only): 17.3061
Epoch 7/30 - VAE train loss (normal only): 16.8417
Epoch 8/30 - VAE train loss (normal only): 16.6168
Epoch 9/30 - VAE train loss (normal only): 16.3493
Epoch 10/30 - VAE train loss (normal only): 15.9950
Epoch 11/30 - VAE train loss (normal only): 15.8055
Epoch 12/30 - VAE train loss (normal only): 15.6443
Epoch 13/30 - VAE train loss (normal only): 15.4159
Epoch 14/30 - VAE train loss (normal only): 15.1695
Epoch 15/30 - VAE train loss (normal only): 14.9287
Epoch 16/30 - VAE train loss (normal only): 14.7496
Epoch 17/30 - VAE train loss (normal only): 14.5148
Epoch 18/30 - VAE train loss (normal only): 14.3917
Epoch 19/30 - VAE train loss (normal only): 14.2309
Epoch 20/30 - VAE tra

In [40]:
# Reconstruction error for ALL data
vae.eval()

X_all_tensor = torch.tensor(X_all_scaled, dtype=torch.float32).to(device)

with torch.no_grad():
    x_recon_all, mu_all, logvar_all = vae(X_all_tensor)
    # per-sample MSE across features
    recon_error = torch.mean(
        (x_recon_all - X_all_tensor) ** 2,
        dim=1
    ).cpu().numpy()

# Attach to df
df['recon_error'] = recon_error

### Finding the best threshold [Maximize F1-score (best for imbalance)]

In [42]:
from sklearn.metrics import precision_recall_curve, f1_score
# True labels
y_true = df['churn'].values

# Scores (reconstruction error)
scores = df['recon_error'].values

prec, rec, thresholds = precision_recall_curve(y_true, scores)

f1_scores = 2 * (prec * rec) / (prec + rec + 1e-9)  # avoid divide by zero
best_idx = np.argmax(f1_scores)
best_threshold_f1 = thresholds[best_idx]

print("Best Threshold (F1):", best_threshold_f1)

Best Threshold (F1): 0.19873132


In [43]:
best_threshold = 0.19873132
df['is_anomaly'] = df['recon_error'] > best_threshold

In [44]:
from sklearn.metrics import confusion_matrix, classification_report

y_true = df['churn']
y_pred = df['is_anomaly'].astype(int)

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Confusion Matrix:
[[3610 9577]
 [ 340 1079]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.27      0.42     13187
           1       0.10      0.76      0.18      1419

    accuracy                           0.32     14606
   macro avg       0.51      0.52      0.30     14606
weighted avg       0.83      0.32      0.40     14606



In [76]:
torch.save(vae.state_dict(), "vae_normal_model.pth")
print("Saved VAE model to vae_normal_model.pth")

Saved VAE model to vae_normal_model.pth


In [77]:
import joblib
joblib.dump(scaler, "vae_normal_scaler.pkl")
print("Saved scaler to vae_normal_scaler.pkl")

Saved scaler to vae_normal_scaler.pkl


In [78]:
import pickle

with open("vae_threshold.pkl", "wb") as f:
    pickle.dump(best_threshold, f)

print("Saved threshold to vae_threshold.pkl")

Saved threshold to vae_threshold.pkl


In [ ]:
# import json
# with open("vae_metadata.json", "w") as f:
#     json.dump({"feature_names": list(X.columns)}, f)


In [45]:
# print(df[['churn', 'recon_error', 'is_anomaly']].head())

# # How many anomalies?
# print(df['is_anomaly'].value_counts())

# # How does anomaly flag align with churn?
# print(pd.crosstab(df['churn'], df['is_anomaly'], rownames=['churn'], colnames=['is_anomaly']))

### VAE synthetic data generation

In [55]:
X = df.drop(columns=['churn'])
y = df['churn'].astype(int)

# minority = churn = 1
X_min = X[y == 1]
X_maj = X[y == 0]

print("Minority (churn=1):", len(X_min))
print("Majority (churn=0):", len(X_maj))

Minority (churn=1): 1419
Majority (churn=0): 13187


In [56]:
scaler_min = StandardScaler()
X_min_scaled = scaler_min.fit_transform(X_min.values.astype(np.float32))

# Tensor + DataLoader
X_min_tensor = torch.tensor(X_min_scaled, dtype=torch.float32)
min_dataset = TensorDataset(X_min_tensor)
min_loader = DataLoader(min_dataset, batch_size=64, shuffle=True)

input_dim = X.shape[1]   # 19 features
hidden_dim = 16
latent_dim = 8

In [57]:
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        # Decoder
        self.fc_dec1 = nn.Linear(latent_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()

    def encode(self, x):
        h = self.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.relu(self.fc_dec1(z))
        return self.fc_out(h)  # linear output

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE(input_dim, hidden_dim, latent_dim).to(device)

optimizer = optim.Adam(vae.parameters(), lr=1e-3)
recon_loss_fn = nn.MSELoss(reduction="sum")

In [59]:
def vae_loss(x, x_recon, mu, logvar):
    recon_loss = recon_loss_fn(x_recon, x)
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss

In [60]:
n_epochs = 30

vae.train()
for epoch in range(1, n_epochs + 1):
    total_loss = 0.0
    for (xb,) in min_loader:
        xb = xb.to(device)

        optimizer.zero_grad()
        x_recon, mu, logvar = vae(xb)
        loss = vae_loss(xb, x_recon, mu, logvar)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(min_dataset)
    print(f"Epoch {epoch}/{n_epochs} - VAE loss (minority): {avg_loss:.4f}")

Epoch 1/30 - VAE loss (minority): 22.9378
Epoch 2/30 - VAE loss (minority): 21.9869
Epoch 3/30 - VAE loss (minority): 21.6346
Epoch 4/30 - VAE loss (minority): 21.3820
Epoch 5/30 - VAE loss (minority): 21.2588
Epoch 6/30 - VAE loss (minority): 21.1282
Epoch 7/30 - VAE loss (minority): 20.9913
Epoch 8/30 - VAE loss (minority): 20.9506
Epoch 9/30 - VAE loss (minority): 20.7973
Epoch 10/30 - VAE loss (minority): 20.6641
Epoch 11/30 - VAE loss (minority): 20.4132
Epoch 12/30 - VAE loss (minority): 20.1013
Epoch 13/30 - VAE loss (minority): 19.8659
Epoch 14/30 - VAE loss (minority): 19.8456
Epoch 15/30 - VAE loss (minority): 19.4341
Epoch 16/30 - VAE loss (minority): 19.2421
Epoch 17/30 - VAE loss (minority): 19.0240
Epoch 18/30 - VAE loss (minority): 18.9352
Epoch 19/30 - VAE loss (minority): 18.6903
Epoch 20/30 - VAE loss (minority): 18.5289
Epoch 21/30 - VAE loss (minority): 18.3513
Epoch 22/30 - VAE loss (minority): 18.1333
Epoch 23/30 - VAE loss (minority): 18.1021
Epoch 24/30 - VAE lo

In [61]:
n_min = len(X_min)
n_maj = len(X_maj)
n_synth = max(0, n_maj - n_min)   # make minority ~ majority

print("Synthetic samples to generate:", n_synth)

vae.eval()
with torch.no_grad():
    z = torch.randn(n_synth, latent_dim).to(device)
    x_synth_scaled = vae.decode(z)
    x_synth = x_synth_scaled.cpu().numpy()

# Inverse scale back to original feature space
x_synth_original = scaler_min.inverse_transform(x_synth)

Synthetic samples to generate: 11768


In [62]:
# 6. Build synthetic DataFrame and fix dtypes
df_synth = pd.DataFrame(x_synth_original, columns=X.columns)

# Columns that should be integers in your data
int_cols = [
    'cons_gas_12m',
    'cons_last_month',
    'has_gas',
    'nb_prod_act',
    'num_years_antig',
    'days_to_end',
    'days_since_modif',
    'days_to_renewal',
    'is_high_power',
    'is_low_usage'
]

# Round and cast to int
df_synth[int_cols] = df_synth[int_cols].round().astype('int64')

# Clip binary columns to {0,1}
for col in ['has_gas', 'is_high_power', 'is_low_usage']:
    df_synth[col] = df_synth[col].clip(0, 1)

# Add churn = 1 label for all synthetic rows
df_synth['churn'] = 1

print("Synthetic minority data shape:", df_synth.shape)
print(df_synth.head())

Synthetic minority data shape: (11768, 22)
   cons_gas_12m  cons_last_month  forecast_cons_12m  forecast_discount_energy  \
0          5271            -3905        2017.832520                  1.645967   
1         -1085            -1153         444.469116                  0.487528   
2         -9046              997        1176.330811                 -0.474848   
3         -7591             2316        1285.114746                  0.813809   
4         39474            16572        4000.092041                  0.728357   

   forecast_meter_rent_12m  has_gas  nb_prod_act  num_years_antig    pow_max  \
0                63.352768        0            1                5  19.656250   
1                24.992004        0            1                5  11.648237   
2                34.714737        0            1                5  13.025222   
3                43.885296        0            1                5  15.978404   
4               151.220703        0            1                5  37.

In [63]:
df_synth.head()

,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,has_gas,nb_prod_act,num_years_antig,pow_max,days_to_end,...,last_month_vs_avg,gas_to_elec_ratio,margin_ratio,profitability_index,is_high_power,is_low_usage,latest_price_off_peak_fix,recon_error,is_anomaly,churn
0,5271,-3905,2017.832520,1.645967,63.352768,0,1,5,19.656250,-176,...,0.578063,11.552348,0.927101,-12.227868,0,1,44.416973,-5.370505,0.680677,1
1,-1085,-1153,444.469116,0.487528,24.992004,0,1,5,11.648237,-150,...,0.793125,-37.141006,1.018136,-0.256764,0,1,44.140812,-6.187146,0.605784,1
2,-9046,997,1176.330811,-0.474848,34.714737,0,1,5,13.025222,-127,...,0.602918,-282.529541,0.948019,1.320397,0,1,44.447121,5.875952,0.562457,1
3,-7591,2316,1285.114746,0.813809,43.885296,0,1,5,15.978404,-149,...,0.820436,47.834023,0.999517,1.793468,0,1,43.611870,3.528708,0.654229,1
4,39474,16572,4000.092041,0.728357,151.220703,0,1,5,37.797756,-148,...,0.986095,-135.125793,1.023592,-5.352486,1,0,41.055923,-1.273744,0.718846,1


In [64]:
df_synth['churn'].value_counts()

churn
1    11768
Name: count, dtype: int64

In [65]:
df_balanced = pd.concat([df, df_synth], ignore_index=True)

In [66]:
df_balanced.head()

,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_discount_energy,forecast_meter_rent_12m,has_gas,nb_prod_act,num_years_antig,pow_max,days_to_end,...,last_month_vs_avg,gas_to_elec_ratio,margin_ratio,profitability_index,is_high_power,is_low_usage,latest_price_off_peak_fix,churn,recon_error,is_anomaly
0,54946,0,0.00,0.0,1.78,1,2,3,43.648,-199,...,0.000000,54946.0,1.0,678.990000,1,1,44.266930,1,207.782501,1.0
1,0,0,189.95,0.0,16.27,0,1,6,13.800,-123,...,0.000000,0.0,1.0,0.004053,0,1,44.444710,0,0.283753,1.0
2,0,0,47.96,0.0,38.72,0,1,6,13.856,-259,...,0.000000,0.0,1.0,0.012110,0,1,44.444710,0,0.230280,1.0
3,0,0,240.04,0.0,19.83,0,1,6,13.200,-276,...,0.000000,0.0,1.0,0.016063,0,1,44.444710,0,0.195203,0.0
4,0,526,445.75,0.0,131.73,0,1,6,19.800,-299,...,1.426441,0.0,1.0,0.010840,1,1,40.728885,0,0.331751,1.0


In [67]:
df_balanced['churn'].value_counts()

churn
1    13187
0    13187
Name: count, dtype: int64

### Now we can fit models on this balanced data 

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, roc_auc_score
)

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

In [69]:
# df_balanced already contains churn + synthetic data
X = df_balanced.drop(columns=['churn'])
y = df_balanced['churn'].astype(int)

# Train-test split (stratified even after balancing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (21099, 21) Test: (5275, 21)


### Random Forest

In [70]:
rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    class_weight=None,   # balanced dataset → no need
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

print("\n=== RANDOM FOREST ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_rf))


=== RANDOM FOREST ===
Confusion Matrix:
[[2637    1]
 [ 257 2380]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      2638
           1       1.00      0.90      0.95      2637

    accuracy                           0.95      5275
   macro avg       0.96      0.95      0.95      5275
weighted avg       0.96      0.95      0.95      5275

ROC-AUC: 0.966201153296688


In [73]:
joblib.dump(rf, "rf_balanced.pkl")
print("Saved: rf_balanced.pkl")

Saved: rf_balanced.pkl


### Logistic Regression 

In [71]:
logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        max_iter=2000,
        solver="liblinear",
        class_weight=None
    ))
])

logreg.fit(X_train, y_train)

y_pred_lr = logreg.predict(X_test)
y_proba_lr = logreg.predict_proba(X_test)[:, 1]

print("\n=== LOGISTIC REGRESSION ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_lr))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_lr))



=== LOGISTIC REGRESSION ===
Confusion Matrix:
[[1640  998]
 [ 900 1737]]

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.62      0.63      2638
           1       0.64      0.66      0.65      2637

    accuracy                           0.64      5275
   macro avg       0.64      0.64      0.64      5275
weighted avg       0.64      0.64      0.64      5275

ROC-AUC: 0.7028380747184682


In [74]:
joblib.dump(logreg, "logreg_balanced.pkl")
print("Saved: logreg_balanced.pkl")

Saved: logreg_balanced.pkl


### Support Vector Machine 

In [72]:
svc = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(
        kernel="rbf",
        probability=True,
        class_weight=None
    ))
])

svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)
y_proba_svc = svc.predict_proba(X_test)[:, 1]

print("\n=== SVC (RBF) ===")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svc))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svc))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_svc))


=== SVC (RBF) ===
Confusion Matrix:
[[2635    3]
 [ 269 2368]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      2638
           1       1.00      0.90      0.95      2637

    accuracy                           0.95      5275
   macro avg       0.95      0.95      0.95      5275
weighted avg       0.95      0.95      0.95      5275

ROC-AUC: 0.9482613579483429


In [75]:
joblib.dump(svc, "svc_balanced.pkl")
print("Saved: svc_balanced.pkl")

Saved: svc_balanced.pkl
